# Análisis de datos de nosepoke

## 0. Importación de librerías

In [191]:
import pandas as pd
import os
import docx2txt
from datetime import datetime, timedelta
import re

In [192]:
path="D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE/"
pathMachos=f"{path}/MACHOS/"
pathHembras=f"{path}/HEMBRAS/"

## 1. Extraer lista de los registros

Lo primero que se hará es extraer la lista de los archivos que hay disponibles. Para esto, se recorrerán de forma iterativa las carpetas donde están los datos almacenados

In [193]:
def listaDeRegistros(base):
    """
    Toma una ubicación y realiza un proceso iterativo buscando en todos los 
    archivos que se encuentren almacenados
    
    Parameters:
    base (str): dirección del directorio base donde se buscarán los archivos que
    haya presentes

    Returns:
    registros (Array of str): lista de registros presentes en la carpeta

   """
    registros=[]
    for i in os.listdir(base):
        if os.path.isdir(base+i):
            registros+=listaDeRegistros(base+i+"/")
        elif os.path.isfile(base+i):
            registros.append(base+i)
    return registros

In [194]:
def listaDeAnimales(registros, sexChar):
    """
    Toma el nombre de archivo de un registro y la divide, extrayendo los datos
    correspondientes, asumiendo que cumple con la convención de formato
    [DD].[MM].[AAAA].[Sex]_C[#Caja]_[#Animal]_[IdPrueba]_D[#dia]_B[#Bloque].[optional].docx
    
    Parameters:
    registros (Array of str): lista de registros
    sexo (Char): Caracter que puede ser 'H' para hembras y 'M' para machos

    Returns:
    data (Pandas dataframe): dataframe de pandas con los datos de cada registro
   """
   
    dia=[]
    mes=[]
    año=[]
    sexo=[]
    caja=[]
    marca=[]
    tipoPrueba=[]
    diaEnsayo=[]
    bloque=[]
    nombreArchivo=[]
    etiqueta=[]
    
    for registro in registros:
        try:
            # Ejemplo de registro '3.12.2021.H_C21_5_REV_AZ_D8_B1.docx'
            direccion=registro.split("/")[-1:][0].split(".")
            
            # '3' '12' '2021'
            dia.append(int(direccion[0]))
            mes.append(int(direccion[1]))
            año.append(int(direccion[2]))
            
            # 'H_C21_5_REV_AZ_D8_B1'
            metadata=[]
            for i in direccion[3].split('_'):
                i = i.strip()
                if ' ' in i.strip():
                    for j in i.split(' '):
                        if j!='':
                            metadata.append(j)
                else:
                    metadata.append(i)
                        
            
            if sexChar in metadata[0]:
                sexo.append(metadata.pop(0).strip())
            else:
                sexo.append(sexChar)
                # print("Error al leer el sexo del animal: "+registro)
                
            if 'C' in metadata[0]:
                caja.append(metadata.pop(0).strip()[1:])
            else:
                caja.append(-1)
                print("Error al leer el identificador de la caja: "+registro)
                
            try:
                int(metadata[0])
                marca.append(metadata.pop(0).strip())
            except:
                marca.append(-1)
                print("Error al leer el numero del animal: "+registro)
                
            if 'B' in metadata[-1]:
                bloque.append(metadata.pop().strip()[1:])
            else:
                bloque.append(-1)
                print("Error al leer el bloque: "+registro)
                
            if 'D' in metadata[-1]:
                diaEnsayo.append(metadata.pop().strip()[1:])
            else:
                diaEnsayo.append(-1)
                print("Error al leer el bloque: "+registro)
            
            tipo=""
            for i in metadata:
                tipo+=(i.strip()+'_')
            tipoPrueba.append(tipo.strip()[:-1])
            
            if len(direccion)==6:
                etiqueta.append(direccion[4].strip())
            elif re.search('ESTANDARI', registro, re.IGNORECASE):
                etiqueta.append('estandarizacion')
            else:
                etiqueta.append('')
            
            nombreArchivo.append(registro)
            
        except:
            print("Error adicional: "+registro)
    d={'dia':dia, 'mes':mes, 'año':año, 'sexo':sexo, 'caja':caja,
       'marca':marca, 'tipoPrueba':tipoPrueba, 'diaEnsayo':diaEnsayo,
       'bloque':bloque, "nombreArchivo":nombreArchivo, 'etiqueta':etiqueta}
    data=pd.DataFrame(data=d)
    return data

In [195]:
if(re.search('ESTANDARI', 'asf/EStandarizacion/asfasdfasfas.csv', re.IGNORECASE)):
    print('A')

A


In [196]:
def estandarizarTerminosTipoDePrueba(data):
    """
    Se tienen variaciones del identificador usado para los 7 tipos de prueba
    que se realizaron. Este método unifica los diferentes términos usados
    
    Parameters:
    data (Pandas dataframe): dataframe de pandas con los datos de cada registro

    Returns:
    cleanData (Pandas dataframe): dataframe de pandas con los términos unificados para la columna tipoPrueba
    """
    cleanData=data.copy()
    replaceDict={"_HAB":"HAB",
                 "P_DER":"SM_ADQ_DER", "SM_P_DER":"SM_ADQ_DER",
                 "SM_PRE_DER":"SM_ADQ_DER", "SM_ADQ_DER_":"SM_ADQ_DER","SM_DER_ADQ":"SM_ADQ_DER",
                 "SMM_ADQ_DER":"SM_ADQ_DER",
                 "P_IZQ":"SM_ADQ_IZQ","SM_P_IZQ":"SM_ADQ_IZQ","SM_IZQ_ADQ":"SM_ADQ_IZQ",
                 "SM_IZQ":"SM_ADQ_IZQ","SM_PRE_IZQ":"SM_ADQ_IZQ",
                 
                 "R_DER":"SM_REV_DER","SM_R_DER":"SM_REV_DER",
                 "R_IZQ":"SM_REV_IZQ","SM_R_IZQ":"SM_REV_IZQ",
                 
                 "DIS":"RA_ADQ", "DIS_AZ":"RA_ADQ", "ADQ_AZ":"RA_ADQ",
                 
                 "REV_AZ":"RA_REV", "RA":"RA_REV", "REV":"RA_REV", "R_AZ":"RA_REV",
                 }
    cleanData['tipoPrueba'].replace(to_replace=replaceDict, inplace=True)
    return cleanData

In [198]:
registrosHembras=estandarizarTerminosTipoDePrueba(listaDeAnimales(listaDeRegistros(pathHembras), 'H'))

Error adicional: D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//HEMBRAS/ACTUALIZADA. REGISTRO APRENDIZAJE EXPERIMENTO -iMac de Diego.xlsx
Error adicional: D:/Users/USER/OneDrive - Universidad de los Andes/REGISTRO APRENDIZAJE//HEMBRAS/Cronograma Actividades Experimento.xlsx


In [199]:
registrosHembras.head()

,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,27,8,2021,H,51,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,27,8,2021,H,51,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
2,27,8,2021,H,51,2,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
3,27,8,2021,H,51,2,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
4,27,8,2021,H,51,3,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,


In [200]:
registrosMachos=estandarizarTerminosTipoDePrueba(listaDeAnimales(listaDeRegistros(pathMachos), 'M'))

In [201]:
registrosMachos.head()

,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,13,8,2021,M,10,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,13,8,2021,M,10,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
2,13,8,2021,M,10,2,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
3,13,8,2021,M,10,2,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
4,13,8,2021,M,12,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,MD


In [202]:
registrosUnificados=pd.concat([registrosMachos, registrosHembras], ignore_index=True)
registrosUnificados.insert(loc=0, column='IdAnimal', value=registrosUnificados.apply(lambda fila: f"{fila.sexo}-{fila.caja}-{fila.marca}", axis=1))
registrosUnificados.to_excel("registrosMachosYHembras.xlsx")

In [203]:
registrosUnificados.head()

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,M-10-1,13,8,2021,M,10,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
1,M-10-1,13,8,2021,M,10,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
2,M-10-2,13,8,2021,M,10,2,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,
3,M-10-2,13,8,2021,M,10,2,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,
4,M-12-1,13,8,2021,M,12,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,MD


### 1.1. Análisis de calidad de los registros

Revisaremos los valores de cada columna para identificar la calidad de los datos

In [204]:
for column in registrosUnificados:
    if column != 'nombreArchivo':
        print({column:registrosUnificados[column].unique()})

{'IdAnimal': array(['M-10-1', 'M-10-2', 'M-12-1', 'M-12-2', 'M-11-1', 'M-11-2',
       'M-11-3', 'M-11-4', 'M-5B6-1', 'M-5B6-4', 'M-5B6-5', 'M-14-1',
       'M-14-2', 'M-21-1', 'M-21-2', 'M-21-3', 'M-24-3', 'M-24-4',
       'M-25-1', 'M-25-3', 'M-25-4', 'M-27-3', 'M-27-4', 'M-31-3',
       'M-31-4', 'M-22-3', 'M-22-4', 'M-23-1', 'M-23-2', 'M-26-1',
       'M-26-2', 'M-28-1', 'M-28-5', 'M-33-4', 'M-33-5', 'M-34-1',
       'M-34-6', 'M-29-5', 'M-29-6', 'M-29-7', 'M-32-1', 'M-32-2',
       'M-32-3', 'M-30-1', 'M-30-2', 'M-30-3', 'M-35-2', 'M-35-4',
       'M-17-1', 'M-19-1', 'M-19-3', 'M-20-2', 'M-20-3', 'M-13-1',
       'M-13-2', 'M-15-1', 'M-15-2', 'M-16-1', 'M-16-2', 'M-17-4',
       'M-17-2', 'H-51-1', 'H-51-2', 'H-51-3', 'H-55-1', 'H-55-2',
       'H-55-3', 'H-56-4', 'H-56-5', 'H-56-6', 'H-56-7', 'H-18-1',
       'H-18-2', 'H-18-3', 'H-10-1', 'H-10-2', 'H-10-3', 'H-11-4',
       'H-11-5', 'H-11-6', 'H-20-1', 'H-20-2', 'H-20-3', 'H-21-4',
       'H-21-5', 'H-21-6', 'H-13-1', 'H-13-2',

In [205]:
for column in registrosMachos:
    if column != 'nombreArchivo':
        print({column:registrosMachos[column].unique()})

{'dia': array([13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
       31,  1, 10, 11, 12, 18,  2,  3,  4,  5,  6,  7,  8,  9],
      dtype=int64)}
{'mes': array([ 8, 12,  1,  2, 11, 10,  9], dtype=int64)}
{'año': array([2021, 2022], dtype=int64)}
{'sexo': array(['M'], dtype=object)}
{'caja': array(['10', '12', '11', '5B6', '14', '21', '24', '25', '27', '31', '22',
       '23', '26', '28', '33', '34', '29', '32', '30', '35', '17', '19',
       '20', '13', '15', '16'], dtype=object)}
{'marca': array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)}
{'tipoPrueba': array(['HAB', 'SM_ADQ_DER', 'SM_ADQ_IZQ', 'SM_REV_DER', 'SM_REV_IZQ',
       'RA_ADQ', 'RA_REV'], dtype=object)}
{'diaEnsayo': array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '17', '11',
       '19', '12', '13', '14', '15', '16', '18', '20', '21', '22', '23',
       '24'], dtype=object)}
{'bloque': array(['1', '2'], dtype=object)}
{'etiqueta': array(['', 'MD', 'TIEMPO', 'estandarizacion', 'estanta

In [206]:
registrosUnificados.query('etiqueta == "estandarizacion"')

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
176,M-5B6-1,25,8,2021,M,5B6,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
177,M-5B6-1,25,8,2021,M,5B6,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
178,M-5B6-4,25,8,2021,M,5B6,4,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
179,M-5B6-4,25,8,2021,M,5B6,4,SM_ADQ_IZQ,1,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
180,M-5B6-5,25,8,2021,M,5B6,5,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
...,...,...,...,...,...,...,...,...,...,...,...,...
2676,M-5B6-5,25,9,2021,M,5B6,5,RA_REV,10,2,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
2681,M-5B6-5,26,9,2021,M,5B6,5,RA_REV,11,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
2682,M-5B6-5,26,9,2021,M,5B6,5,RA_REV,11,2,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
2687,M-5B6-5,28,9,2021,M,5B6,5,RA_REV,12,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion


In [207]:
registrosUnificados.query('etiqueta == "estandarizacion"')['IdAnimal'].unique()

array(['M-5B6-1', 'M-5B6-4', 'M-5B6-5', 'M-14-1'], dtype=object)

In [208]:
registrosUnificados.query('etiqueta == "" & IdAnimal == "M-5B6-4"')

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta


In [209]:
# Los archivos del 25 y 26 de agosto no están dentro de los archivos
registrosUnificados.query('IdAnimal == "M-22-3" & tipoPrueba == "RA_REV"').head(70)

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
1143,M-22-3,11,1,2022,M,22,3,RA_REV,4,1,D:/Users/USER/OneDrive - Universidad de los An...,
1171,M-22-3,12,1,2022,M,22,3,RA_REV,5,1,D:/Users/USER/OneDrive - Universidad de los An...,
1198,M-22-3,13,1,2022,M,22,3,RA_REV,6,1,D:/Users/USER/OneDrive - Universidad de los An...,
1223,M-22-3,14,1,2022,M,22,3,RA_REV,7,1,D:/Users/USER/OneDrive - Universidad de los An...,
1249,M-22-3,15,1,2022,M,22,3,RA_REV,8,1,D:/Users/USER/OneDrive - Universidad de los An...,
1275,M-22-3,17,1,2022,M,22,3,RA_REV,9,1,D:/Users/USER/OneDrive - Universidad de los An...,
1303,M-22-3,18,1,2022,M,22,3,RA_REV,10,1,D:/Users/USER/OneDrive - Universidad de los An...,
1304,M-22-3,18,1,2022,M,22,3,RA_REV,10,2,D:/Users/USER/OneDrive - Universidad de los An...,
1347,M-22-3,19,1,2022,M,22,3,RA_REV,11,1,D:/Users/USER/OneDrive - Universidad de los An...,
1348,M-22-3,19,1,2022,M,22,3,RA_REV,11,2,D:/Users/USER/OneDrive - Universidad de los An...,


In [210]:
registrosUnificados.query('caja == "14" & sexo=="M"')

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
288,M-14-1,31,8,2021,M,14,1,RA_REV,3,1,D:/Users/USER/OneDrive - Universidad de los An...,estandarizacion
289,M-14-2,31,8,2021,M,14,2,RA_REV,7,1,D:/Users/USER/OneDrive - Universidad de los An...,estantadarizacion


Los datos de la columna 'etiqueta' indican errores que se encontraron luego de una revisión manual

| Etiqueta | Descripción |
| --- | --- |
| MD | Datos perdidos, están en blanco |
| TIME/TIEMPO | No tienen el registro del tiempo |
| .resumen_final | Solo arrojaron el resumen de todo el bloque (ensayo 50) |
| SPLIT/Split/split | Archivos que tienen entre 30 y 50 ensayos, los cuales no son consecutivos |
| EXCLUDE |  Machos C35_2 Y C21_2. Uno de ellos no logró el criterio de habituación en 20 días y debimos sacarlo del estudio, mientras que el otro se murió por el camino. De ese animal solo tenemos los datos de habituación que pueden servir (C35_2) |
| error de asignación | Se excluyen dado que se hizo otro protocolo al que correspondía ese día |
| estandarizacion | Se exlcuyen del análisis |

In [211]:
replaceDict={"Split":"SPLIT",
             "split":"SPLIT",
             "TIME":"TIEMPO"
                 }
registrosUnificados['etiqueta'].replace(to_replace=replaceDict, inplace=True)

In [212]:
# registrosUnificados.loc[registrosUnificados['etiqueta'] == 'SPLIT']['nombreArchivo'].values
registrosUnificados.loc[registrosUnificados['etiqueta'] == 'SPLIT'].head(5)

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
330,M-23-1,10,12,2021,M,23,1,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
346,M-27-4,11,12,2021,M,27,4,HAB,1,2,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
383,M-23-2,12,12,2021,M,23,2,HAB,3,1,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
396,M-27-3,13,12,2021,M,27,3,RA_ADQ,1,2,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT
397,M-27-3,13,12,2021,M,27,3,HAB,4,1,D:/Users/USER/OneDrive - Universidad de los An...,SPLIT


### 1.2. Estadísticos generales
Se mirará el número de datos para cada tipo de prueba en machos y hembras

In [213]:
# Número de archivos por cada tipo de prueba en machos
registrosMachos['tipoPrueba'].value_counts()

RA_REV        1365
HAB            456
RA_ADQ         331
SM_REV_DER     205
SM_ADQ_IZQ     139
SM_REV_IZQ     117
SM_ADQ_DER      84
Name: tipoPrueba, dtype: int64

In [214]:
idMachos = registrosUnificados.query('sexo == "M"')['IdAnimal'].unique()

In [215]:
filesPerAnimal = {}
for id in idMachos:
    fileCount = {}
    countByType = registrosUnificados.query(f'IdAnimal == "{id}" & (etiqueta == "" | etiqueta == "TIEMPO" | etiqueta == "SPLIT")')['tipoPrueba'].value_counts().rename_axis('unique_values').reset_index(name='counts')
    for fileType in countByType['unique_values'].unique():
        fileCount[fileType] = countByType['counts'][countByType['unique_values'] == fileType ].values[0]
    filesPerAnimal[id] = fileCount

In [217]:
pd.DataFrame.from_dict(filesPerAnimal, orient='index').to_excel('Número de archivos por tipo de prueba machos.xlsx')

## 2. Extraer datos de cada tipo de prueba
Vamos a extraer los datos de cada tipo de prueba. Para esto definiremos el enabezado de cada tipo de prueba que vamos a revisar y leeremos cada archivo.

¡OJO! El máximo número de bloques a analizar es 14 por animal

In [14]:
medidas={
    "HAB":{
        "TituloColumna":["Ensayo_", "Aciertos_",
                         "PorcentajeAciertos_","TiempoRespuestasImpulsivas_",
                         "Omisiones_", "PorcentajeOmisiones_", "Categorias_",
                         "CorrectasDerecha_","CorrectasIzquierda_","LatenciaPromedio_",
                         "MaximoAciertosSeguidos_"],
        # 0: Numero, 1: porcentaje, 2: tiempo con "ms", -1: número de ensayo
        "TipoDeDato":[-1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0],
        "EncabezadoStr":"Ensayo,Aciertos,Porcentaje aciertos,Tiempo total en Respuestas impulsivas,Omisiones,Porcentaje omisiones,Categorias,Total correctas a la derecha,Total correctas a la izquierda,Latencia promedio,Maximo de aciertos seguidos"
    },
}
    

In [15]:
def separarLinea(linea, etiqueta):
    """
    Cada línea de los archivos tiene una estructura "HH:MM:SS.MS -> DATOS".
    En caso de que no haya una entrada de tiempo se pone como hora '-1'
    
    Parameters:
    linea (String): linea de texto extraída del archivo de datos
    etiqueta (String): indica si hay más de un registro en el archivo "SPLIT" o si no tiene el tiempo "TIEMPO"

    Returns:
    hora, registro (String, String): hora y datos por separado.
    """
    if len(linea.split('->'))==2:
        hora=linea.split('->')[0]
        registro=linea.split('->')[1]
    else:
        hora=-1
        registro=linea
    return hora, registro

In [ ]:
def extraerDatosEnsayo(texto: str, fileName, diaEnsayo, bloque, idAnimal, tipoPrueba, etiqueta):
    """
    Se carga el registro de cada animal en formato de texto y se realiza la 
    extracción de datos
    
    Parameters:
    texto (String): Cadena de texto con los datos del ensayo
    fileName (String): dirección del archivo de registro
    diaEnsayo (Int): día del ensyo en que se tomaron los datos
    bloque (Int): bloque dentro del día en que se tomaron los datos
    tipoPrueba (String): Cadena de texto con el nombre de la prueba
    etiqueta (String): Indica si hay más de un registro en el archivo "SPLIT" o si no tiene el tiempo "TIEMPO"

    Returns:
    dataDict (Pandas Dataframe): diccionario con los datos del ensayo
    """
    texto=texto.split('\n')
    # Se eliminan líneas vacías
    texto= list(filter(None, texto))
    dataDict={"Dirección":fileName,
              "Tipo de prueba": tipoPrueba,
              "Día":diaEnsayo,
              "Bloque": bloque,
              "IdAnimal":idAnimal}
    
    if tipoPrueba=='SM_ADQ_DER' or tipoPrueba=='SM_ADQ_IZQ':
        tipoPrueba='SM_ADQ'
    
    # Variables para registrar el tiempo
    horaInicio=""
    horaFin=""
    inicioLogged=True
    finLogged=True
    
    # Se usará como contador del número de líneas
    i=0
    
    # Si el archivo está vacío genera un error
    if len(texto)==0:
        print(f'{fileName}, Archivo vacío')
    else:
        # Se lee el archivo línea a línea
        while i < len(texto):
            linea=texto[i]
            # Cada línea se espera que tenga la siguiente estructura "12:20:12.886 -> Transmitiendo"
            hora, registro = separarLinea(linea, etiqueta)
                
            # Registra el primer timestamp del archivo. En caso de que encuentre la marca de inicio lo sobreescribe
            if (registro.find("Van a empezar los 50 ensayos")>-1 or inicioLogged) and hora!=-1:
                horaInicio=datetime.strptime(hora.split('.')[0],"%H:%M:%S")
                inicioLogged=False
                
            # Si los datso tienen la siguiente estructura: "1.00,1,2.00,0.00,0,0.00,0,0,1,8388.00,1"
            elif len(registro.split(','))==len(medidas[tipoPrueba]['TituloColumna']):
                encabezadoOk=False
                while i < len(texto):
                    linea=texto[i]
                    hora, registro = separarLinea(linea)
                    # Se revisará que el archivo tenga el encabezado esperado
                    if medidas[tipoPrueba]['EncabezadoStr'] in registro:
                        encabezadoOk=True
                    else:
                        datos=registro.split(',')
                        numReg=float(datos[0])
                        for j in range( 1, (len(medidas[tipoPrueba]['TituloColumna'])) ):
                            try:
                                nombreColumna=medidas[tipoPrueba]['TituloColumna'][j]
                                # Este dato es un número o una marca de tiempo SIN unidades ("ms")
                                if medidas[tipoPrueba]['TipoDeDato'][j]==0 or medidas[tipoPrueba]['TipoDeDato'][j]==2:
                                    dataDict[nombreColumna+str(numReg)]=[float(datos[j])]
                                # Este dato es un porcentaje
                                elif medidas[tipoPrueba][1][j]==1:
                                    dataDict[nombreColumna+str(numReg)]=[((float(datos[j-1])/numReg)*100)]
                            except:
                                print(fileName +", El tipo de dato no fue el esperado")
                        if numReg == 50:
                            break
                    i+=1
            elif registro.find("Ensayo numero:")>-1:
                numReg=float(registro.split(':')[1])
                
                while True:
                    if registro.find("Aciertos")>-1:
                        break
                    else:
                        i+=1
                        linea=texto[i]
                        hora, registro = separarLinea(linea)
                registroAnterior=-1
                for j in range( 1, (len(medidas[tipoPrueba][0])) ):
                    linea=texto[i]
                    hora, registro = separarLinea(linea)
                    try:
                        nombreColumna=medidas[tipoPrueba][0][j]
                        # Este dato es un número
                        if medidas[tipoPrueba][1][j]==0:
                            dataDict[nombreColumna+str(numReg)]=[float(registro.split(':')[1])]
                            registroAnterior=float(registro.split(':')[1])
                        # Este dato es un porcentaje
                        elif medidas[tipoPrueba][1][j]==1:
                            dataDict[nombreColumna+str(numReg)]=[((registroAnterior/numReg)*100)]
                        # Este dato es una marca de tiempo CON unidades ("ms")
                        elif medidas[tipoPrueba][1][j]==2:
                            data=registro.split(':')[1].strip().split(" ")[0]
                            dataDict[nombreColumna+str(numReg)]=[data]
                    except:
                        logFile.write(fileName +", El tipo de dato no fue el esperado\n")
                    i+=1
                i-=1
            i+=1
        
        try:
            horaFin=datetime.strptime(hora.split('.')[0],"%H:%M:%S")
            segundosTotales=(horaFin-horaInicio).total_seconds()
            if segundosTotales < 0:
                horaFin -= timedelta(hours=1)
                horaInicio -= timedelta(hours=1)
                segundosTotales=(horaFin-horaInicio).total_seconds()
            dataDict["segundosTotales"]=[segundosTotales]
        except:
            dataDict["segundosTotales"]=[-1]
            logFile.write(fileName +", No se puede calcular el tiempo total del experimento\n")
    return pd.DataFrame.from_dict(dataDict)

In [ ]:
def procesarLineaCSV(linea):
    
    

### 2.1. Extraer datos habituación
Cada prueba de habituación cuenta con 50 ensayos. En cada uno de ellos se tienen los siguientes datos:

| Nombre columna | Descripción | Rango |
| --- | --- | --- |
| Ensayo | Número de ensayo | 1 - 50 |
| Aciertos | Número acomulado de aciertos | 0 - 50 |
| Porcentaje aciertos | | 0% - 100% |
| Tiempo total en respuestas impulsivas | Tiempo acomulado que pasa metiendo a la nariz entre ensayos | Continua (segudos) |
| Omisiones | Número acomulado de omisiones | 0 - 50 |
| Porcentaje omisiones | | 0% - 100% |
| Categorias | Cada vez que complete 3 aciertos seguidos se cuenta una categoría| 0 - 16 |
| Total correctas a la derecha | | 0 - 50 |
| Total correctas a la izquierda | | 0 - 50 |
| Latencia promedio | Tiempo promedio en ensayos con aciertos | Contínua (milisegundos) |
| Maximo de aciertos seguidos | | 0- 50 |

Algunos archivos codifican las respuestas impulsivas contándolas y no como el tiempo. Dada la sensibilidad del sensor, esto lleva en algunos casos a números muy elevados.

In [16]:
registrosUnificados.loc[registrosUnificados['tipoPrueba'] == 'HAB'].head(1)

,IdAnimal,dia,mes,año,sexo,caja,marca,tipoPrueba,diaEnsayo,bloque,nombreArchivo,etiqueta
0,M-10-1,13,8,2021,M,10,1,HAB,1,1,D:/Users/USER/OneDrive - Universidad de los An...,


In [ ]:
#Vamos a ver los datos que se tienen por cada animal
outputHabituacion=pd.DataFrame()
for animal in registrosUnificados['IdAnimal'].unique():
    outputAnimal=pd.DataFrame()
    for indice, fila in registrosUnificados[ registrosUnificados['IdAnimal']==animal].iterrows():        
        if fila['etiqueta'] == 'MD' or fila['etiqueta'] == 'EXCLUDE':
            continue
        else:
            try:
                # Datos que queremos guardar de cada registro
                tipoPrueba=fila['tipoPrueba']
                sexo=fila['sexo']
                
                # Convertimos el .docx a texto
                if tipoPrueba == 'HAB' and sexo=="M":
                    text=docx2txt.process(fila['nombreArchivo'])
                    registroSesion=extraerDatosEnsayo(text,
                                                  fila['nombreArchivo'],
                                                  fila['diaEnsayo'],
                                                  fila['bloque'],
                                                  fila['IdAnimal'],
                                                  tipoPrueba,
                                                  fila['etiqueta']
                                                 )
                    output=pd.concat([output, registroSesion],  ignore_index=True)
            except:
                print(f"{fila['nombreArchivo']} Error al leer el archivo")
        
    output.to_excel(animal+".xlsx")
    outputEnsayo=pd.concat([outputEnsayo, output],  ignore_index=True)
outputEnsayo.to_excel("datosHabituacionMachos.xlsx")